## Import the necessary libraries

In [82]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load in your data from kaggle.  
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [83]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

In [84]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [85]:
train['message'] = train['message'].str.lower()

In [ ]:
#train['message']  = train['message'] .str.replace('\d+', '')

In [ ]:
#train['message'] = train['message'].str.replace('&amp','&')  

In [ ]:
#train['message'] = train['message'].str.replace('rt','')  

In [ ]:
#train['message'] = train['message'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
#train['message'] = train['message'].str.replace('#','')  

In [ ]:
#from nltk.tokenize import word_tokenize, TreebankWordTokenizer

In [ ]:
#tokeniser = TreebankWordTokenizer()

#combine['Tidy_Tweets'] = combine['Tidy_Tweets'].apply(tokeniser.tokenize)

In [ ]:
#from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer
#stemmer = PorterStemmer()
#def climate_stemmer(words, stemmer):
#    return [stemmer.stem(word) for word in words]
#train['message'] = train['message'].apply(climate_stemmer, args=(stemmer, ))

In [ ]:
#from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
#lemmatizer = WordNetLemmatizer()
#train['message'] = train['message'].apply(climate_lemma, args=(lemmatizer, ))

In [ ]:
#from nlppreprocess import NLP
#nlp = NLP()

In [ ]:
#def cleaner(line):

#    punctuation = re.compile("[.;:!\'’‘“”?,\"()\[\]]")
#    tweet = punctuation.sub("", line.lower()) 
    # Removes stopwords
#    nlp_for_stopwords = NLP(replace_words=True, remove_stopwords=True, 
#                            remove_numbers=True, remove_punctuations=False) 
#    tweet = nlp_for_stopwords.process(tweet) 

#    return tweet

In [ ]:
#combine['Tidy_Tweets'] = combine['message'].apply(cleaner)

In [24]:
#def remove_symbols(tweet, pattern):
#    r = re.findall(pattern, tweet)
#    for i in r:
#        tweet = re.sub(i, '', tweet)
        
#    return tweet

In [25]:
#train['message'] = np.vectorize(remove_symbols)(train['message'], "@[\w]*") 

In [26]:
train.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesn't think carbon di...,625221
1,1,it's not like we lack evidence of anthropogeni...,126103
2,2,rt : researchers say we have three years to ac...,698562
3,1,#todayinmaker# wired : 2016 was a pivotal year...,573736
4,1,"rt : it's 2016, and a racist, sexist, climate ...",466954


In [28]:
#train['message'] = train['message'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
#from sklearn.utils import resample

In [ ]:
#pro = train[train['sentiment']==1]
#anti = train[train['sentiment']==-1]
#neutral = train[train['sentiment']==0]
#news = train[train['sentiment']==2]

In [ ]:
"""anti_upsampled = resample(anti,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=len(pro), # match number in minority class
                          random_state=27) # reproducible results

neutral_upsampled = resample(neutral,
                             replace=True, # sample with replacement (we need to duplicate observations)
                             n_samples=len(pro), # match number in minority class
                             random_state=27) # reproducible results

news_upsampled = resample(news,
                          replace=True, # sample with replacement (we need to duplicate observations)
                          n_samples=len(pro), # match number in minority class
                          random_state=27) # reproducible results

# Combine upsampled minority class with majority class
upsampled = pd.concat([anti_upsampled, neutral_upsampled,news_upsampled,pro])

# Check new class counts
upsampled.sentiment.value_counts()"""

## Splitting out the X variable from the target

In [86]:
y = train['sentiment']
X = train['message']

## Turning text into something your model can read

In [87]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

## Splitting the training data into a training and validation set

In [88]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.20,shuffle=True, stratify=y, random_state=1)

## Training the model and evaluating using the validation set 

In [89]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_val)

## Checking the performance of our model on the validation set

In [90]:
f1_score(y_val, svc_pred, average="macro")

0.6695910675639931

## Getting our test set ready 

In [91]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [92]:
y_pred = svc.predict(test_vect)

In [93]:
test['sentiment'] = y_pred

In [94]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


## Creating an output csv for submission

In [78]:
test[['tweetid','sentiment']].to_csv('testsubmission7.csv', index=False)